In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install transformers
!pip install torchvision
!pip install tqdm
!pip install open-clip-torch


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-fg691enz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-fg691enz
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 854.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
import clip
from tqdm import tqdm
import warnings
import requests
from io import BytesIO
from transformers import DetrImageProcessor, DetrForObjectDetection
from transformers import BlipProcessor, BlipForConditionalGeneration
import open_clip

warnings.filterwarnings('ignore')

class EnsembleClassifier:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        print(f"Using device: {device}")
        self.models = {}
        self.processors = {}
        self.setup_models()

    def setup_models(self):
        """Initialize multiple pretrained models for ensemble and DETR for object detection"""
        print("Loading ensemble of models...")

        # 1. OpenAI CLIP ViT-B/32
        try:
            print("Loading OpenAI CLIP ViT-B/32...")
            self.models['clip_vit_b32'], self.processors['clip_vit_b32'] = clip.load("ViT-B/32", device=self.device)
            print("✓ OpenAI CLIP ViT-B/32 loaded")
        except Exception as e:
            print(f"✗ Failed to load OpenAI CLIP ViT-B/32: {e}")

        # 2. OpenAI CLIP ViT-L/14
        try:
            print("Loading OpenAI CLIP ViT-L/14...")
            self.models['clip_vit_l14'], self.processors['clip_vit_l14'] = clip.load("ViT-L/14", device=self.device)
            print("✓ OpenAI CLIP ViT-L/14 loaded")
        except Exception as e:
            print(f"✗ Failed to load OpenAI CLIP ViT-L/14: {e}")

        # 3. OpenCLIP ViT-B-32
        try:
            print("Loading OpenCLIP ViT-B-32 (laion2b_s34b_b79k)...")
            self.models['openclip_vit_b32'], _, self.processors['openclip_vit_b32'] = open_clip.create_model_and_transforms(
                'ViT-B-32', pretrained='laion2b_s34b_b79k', device=self.device
            )
            self.tokenizer_openclip = open_clip.get_tokenizer('ViT-B-32')
            print("✓ OpenCLIP ViT-B-32 loaded")
        except Exception as e:
            print(f"✗ Failed to load OpenCLIP: {e}")

        # 4. BLIP model
        try:
            print("Loading BLIP model...")
            self.processors['blip'] = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
            self.models['blip'] = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(self.device)
            print("✓ BLIP model loaded")
        except Exception as e:
            print(f"✗ Failed to load BLIP: {e}")

        # 5. DETR object detection
        try:
            print("Loading DETR object detection model...")
            self.processors['detr'] = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
            self.models['detr'] = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50").to(self.device)
            print("✓ DETR model loaded")
        except Exception as e:
            print(f"✗ Failed to load DETR: {e}")

        print(f"Loaded {len(self.models)} models successfully!")

    def load_image(self, image_url):
        """Load image from URL with better error handling"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(image_url, timeout=20, headers=headers)
            response.raise_for_status()
            image = Image.open(BytesIO(response.content)).convert('RGB')
            return image
        except Exception as e:
            print(f"Error loading image from {image_url}: {str(e)[:100]}")
            return None

    def detect_objects(self, image, confidence_threshold=0.7):
        """Detect objects in image using DETR"""
        try:
            inputs = self.processors['detr'](images=image, return_tensors="pt").to(self.device)
            with torch.no_grad():
                outputs = self.models['detr'](**inputs)

            target_sizes = torch.tensor([image.size[::-1]]).to(self.device)
            results = self.processors['detr'].post_process_object_detection(
                outputs, target_sizes=target_sizes, threshold=confidence_threshold
            )[0]

            detected_objects = []
            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                box = [round(i, 2) for i in box.tolist()]
                label_name = self.models['detr'].config.id2label[label.item()]
                detected_objects.append({
                    'label': label_name,
                    'confidence': round(score.item(), 3),
                    'box': box
                })
            return detected_objects
        except Exception as e:
            print(f"Error during object detection: {str(e)[:100]}")
            return []

    def crop_object_from_image(self, image, box, padding=10):
        """Crop object from image using bounding box with padding"""
        try:
            x_min, y_min, x_max, y_max = box
            width, height = image.size
            x_min = max(0, int(x_min - padding))
            y_min = max(0, int(y_min - padding))
            x_max = min(width, int(x_max + padding))
            y_max = min(height, int(y_max + padding))
            cropped_image = image.crop((x_min, y_min, x_max, y_max))
            return cropped_image
        except Exception as e:
            print(f"Error cropping image: {str(e)[:100]}")
            return None

    def classify_with_clip_openai(self, image, model_name, pickup_classes):
        """Classify using OpenAI CLIP models"""
        try:
            model = self.models[model_name]
            preprocess = self.processors[model_name]
            image_input = preprocess(image).unsqueeze(0).to(self.device)
            text_inputs = clip.tokenize(pickup_classes).to(self.device)
            with torch.no_grad():
                logits_per_image, _ = model(image_input, text_inputs)
                probs = F.softmax(logits_per_image, dim=-1).cpu().numpy()[0]
            return probs
        except Exception as e:
            print(f"Error in {model_name}: {str(e)[:50]}")
            return None

    def classify_with_openclip(self, image, pickup_classes):
        """Classify using OpenCLIP"""
        try:
            model = self.models['openclip_vit_b32']
            preprocess = self.processors['openclip_vit_b32']
            image_input = preprocess(image).unsqueeze(0).to(self.device)
            text_inputs = self.tokenizer_openclip(pickup_classes).to(self.device)
            with torch.no_grad():
                image_features = model.encode_image(image_input)
                text_features = model.encode_text(text_inputs)
                image_features = F.normalize(image_features, dim=-1)
                text_features = F.normalize(text_features, dim=-1)
                logits = (image_features @ text_features.T) * model.logit_scale.exp()
                probs = F.softmax(logits, dim=-1).cpu().numpy()[0]
            return probs
        except Exception as e:
            print(f"Error in OpenCLIP: {str(e)[:50]}")
            return None

    def get_blip_description(self, image):
        """Get image description using BLIP"""
        try:
            processor = self.processors['blip']
            model = self.models['blip']
            inputs = processor(image, return_tensors="pt").to(self.device)
            with torch.no_grad():
                out = model.generate(**inputs, max_length=50)
                description = processor.decode(out[0], skip_special_tokens=True)
            return description
        except Exception as e:
            print(f"Error in BLIP: {str(e)[:50]}")
            return ""

    def classify_with_blip_enhanced(self, image, pickup_classes):
        """Use BLIP description to enhance classification"""
        try:
            description = self.get_blip_description(image)
            if not description:
                return None
            description_lower = description.lower()
            scores = []
            for class_text in pickup_classes:
                key_terms = []
                if "aluminium" in class_text:
                    key_terms = ["aluminum", "aluminium", "metal"]
                elif "battery" in class_text:
                    key_terms = ["battery", "power", "electric"]
                elif "books" in class_text:
                    key_terms = ["book", "paper", "pages"]
                elif "brass" in class_text:
                    key_terms = ["brass", "metal", "golden"]
                elif "ceiling fan" in class_text:
                    key_terms = ["fan", "ceiling", "blade"]
                elif "computer" in class_text or "CPU" in class_text:
                    key_terms = ["computer", "cpu", "desktop", "tower"]
                elif "refrigerator" in class_text or "fridge" in class_text:
                    key_terms = ["refrigerator", "fridge", "cooling"]
                elif "washing machine" in class_text:
                    key_terms = ["washing", "machine", "laundry"]
                elif "TV" in class_text or "television" in class_text:
                    key_terms = ["tv", "television", "screen", "monitor"]
                elif "laptop" in class_text:
                    key_terms = ["laptop", "notebook", "computer"]
                elif "printer" in class_text:
                    key_terms = ["printer", "printing"]
                elif "steel" in class_text:
                    key_terms = ["steel", "metal", "iron"]
                elif "plastic" in class_text:
                    key_terms = ["plastic"]
                else:
                    words = class_text.replace("a photo of", "").replace("a", "").replace("an", "").strip().split()
                    key_terms = [word for word in words if len(word) > 3]
                score = sum(1 for term in key_terms if term in description_lower) / max(len(key_terms), 1)
                scores.append(score)
            scores = np.array(scores)
            if scores.max() > 0:
                scores = scores / scores.sum()
            return scores
        except Exception as e:
            print(f"Error in BLIP-enhanced classification: {str(e)[:50]}")
            return None

    def ensemble_classify(self, image, pickup_classes):
        """Classify using ensemble of models"""
        all_predictions = {}
        valid_predictions = []

        if 'clip_vit_b32' in self.models:
            probs = self.classify_with_clip_openai(image, 'clip_vit_b32', pickup_classes)
            if probs is not None:
                all_predictions['clip_vit_b32'] = probs
                valid_predictions.append(probs)

        if 'clip_vit_l14' in self.models:
            probs = self.classify_with_clip_openai(image, 'clip_vit_l14', pickup_classes)
            if probs is not None:
                all_predictions['clip_vit_l14'] = probs
                valid_predictions.extend([probs, probs])  # Double weight for ViT-L/14

        if 'openclip_vit_b32' in self.models:
            probs = self.classify_with_openclip(image, pickup_classes)
            if probs is not None:
                all_predictions['openclip_vit_b32'] = probs
                valid_predictions.append(probs)

        if 'blip' in self.models:
            probs = self.classify_with_blip_enhanced(image, pickup_classes)
            if probs is not None:
                all_predictions['blip_enhanced'] = probs
                valid_predictions.append(probs * 0.5)  # Lower weight for BLIP

        if not valid_predictions:
            return None, 0.0, all_predictions

        ensemble_probs = np.mean(valid_predictions, axis=0)
        best_idx = np.argmax(ensemble_probs)
        best_class = pickup_classes[best_idx]
        confidence = float(ensemble_probs[best_idx])

        return best_class, confidence, all_predictions

    def filter_relevant_objects(self, detected_objects):
        """Filter detected objects based on relevance to pickup classes"""
        relevant_keywords = [
            'refrigerator', 'laptop', 'computer', 'monitor', 'tv', 'television', 'keyboard',
            'mouse', 'printer', 'scanner', 'microwave', 'washing machine', 'fan', 'chair',
            'scooter', 'air conditioner', 'battery', 'book', 'bottle', 'metal', 'plastic',
            'glass', 'paper', 'electronic', 'appliance', 'wire', 'cable'
        ]
        relevant_objects = [obj for obj in detected_objects if any(keyword in obj['label'].lower() for keyword in relevant_keywords)]
        return relevant_objects

    def process_image(self, image_url, pickup_classes):
        """Complete pipeline: load image, detect objects, classify relevant objects"""
        image = self.load_image(image_url)
        if image is None:
            return None, 0.0, {}, ""

        # Get BLIP description for the whole image
        blip_description = self.get_blip_description(image) if 'blip' in self.models else ""

        # Detect objects
        detected_objects = self.detect_objects(image)
        if not detected_objects:
            print("No objects detected, classifying whole image")
            predicted_class, confidence, model_predictions = self.ensemble_classify(image, pickup_classes)
            return predicted_class, confidence, model_predictions, blip_description

        # Filter relevant objects
        relevant_objects = self.filter_relevant_objects(detected_objects)
        if not relevant_objects:
            print("No relevant objects detected, classifying whole image")
            predicted_class, confidence, model_predictions = self.ensemble_classify(image, pickup_classes)
            return predicted_class, confidence, model_predictions, blip_description

        # Classify each relevant object
        classification_results = []
        for i, obj in enumerate(relevant_objects):
            print(f"Processing detected object {i+1}: {obj['label']} (confidence: {obj['confidence']})")
            cropped_image = self.crop_object_from_image(image, obj['box'])
            if cropped_image is not None:
                predicted_class, confidence, model_predictions = self.ensemble_classify(cropped_image, pickup_classes)
                if predicted_class:
                    classification_results.append({
                        'detected_object': obj['label'],
                        'detection_confidence': obj['confidence'],
                        'predicted_class': predicted_class,
                        'classification_confidence': confidence,
                        'bounding_box': obj['box'],
                        'model_predictions': model_predictions
                    })

        if classification_results:
            best_result = max(classification_results, key=lambda x: x['classification_confidence'])
            return (
                best_result['predicted_class'],
                best_result['classification_confidence'],
                best_result['model_predictions'],
                blip_description
            )

        # Fallback to whole image classification if no valid object classifications
        print("No valid object classifications, classifying whole image")
        predicted_class, confidence, model_predictions = self.ensemble_classify(image, pickup_classes)
        return predicted_class, confidence, model_predictions, blip_description

def main():
    pickup_classes = [
          "a photo of an air conditioner",
          "a photo of an outdoor air conditioner unit",
          "a photo of air conditioner grill scrap",
          "a photo of a split air conditioner",
          "a photo of a window air conditioner",
          "a photo of aluminum sheet scrap",
          "a photo of aluminum utensils scrap",
          "a photo of aluminum scrap",
          "a photo of aluminum parts",
          "a photo of a used inverter battery",
          "a photo of a dry battery",
          "a photo of a bicycle",
          "a photo of books",
          "a photo of brass scrap",
          "a photo of a CD case",
          "a photo of a computer CPU",
          "a photo of a CRT monitor",
          "a photo of cardboard boxes",
          "a photo of duplex cardboard",
          "a photo of raw cardboard",
          "a photo of a ceiling fan",
          "a photo of a plastic chair",
          "a photo of a plastic chair",
          "a photo of clothes",
          "a photo of iron scrap",
          "a photo of a set-top box",
          "a photo of air conditioner compressor scrap",
          "a photo of copper wires",
          "a photo of a desktop computer",
          "a photo of a DVD player or set-top box",
          "a photo of a deep freezer",
          "a photo of a double-door refrigerator",
          "a photo of a refrigerator door",
          "a photo of electronic e-waste",
          "a photo of a heating filament or element",
          "a photo of a refrigerator",
          "a photo of a front-load washing machine",
          "a photo of a fully automatic front-load washing machine",
          "a photo of a fully automatic top-load washing machine",
          "a photo of a copper water heater",
          "a photo of an iron or steel geyser",
          "a photo of broken glass",
          "a photo of glass bottles",
          "a photo of an inverter battery",
          "a photo of an electric iron",
          "a photo of a laptop charger",
          "a phot of charger or adapter",
          "a photo of a metal cooler",
          "a photo of an exhaust fan",
          "a photo of an LCD or LED screen",
          "a photo of a laptop",
          "a photo of a microwave oven",
          "a photo of mixed paper waste",
          "a photo of assorted plastic items",
          "a photo of mixed iron items",
          "a photo of aluminum-dominant scrap (90%)",
          "a photo of brass-dominant scrap (90%)",
          "a photo of iron-dominant scrap (90%)",
          "a photo of steel-dominant scrap (90%)",
          "a photo of a mixer or grinder",
          "a photo of a touchscreen mobile phone",
          "a photo of a TV or monitor",
          "a photo of an electric motor",
          "a photo of a fan motor or wall fan",
          "a photo of old newspapers",
          "a photo of a mobile phone",
          "a photo of assorted plastic",
          "a photo of a plastic air cooler",
          "a photo of household plastic appliances",
          "a photo of a plastic fan",
          "a photo of a plastic sheet",
          "a photo of a printer or scanner",
          "a photo of office electronics (fax, printer, LCD, LED)",
          "a photo of an RO water purifier",
          "a photo of a router or modem",
          "a photo of a scooter",
          "a photo of a scooty",
          "a photo of a semi-automatic washing machine",
          "a photo of a single-door refrigerator",
          "a photo of small iron appliances",
          "a photo of small plastic appliances",
          "a photo of a power stabilizer",
          "a photo of an inverter",
          "a photo of a UPS unit",
          "a photo of stainless steel",
          "a photo of steel grade 304",
          "a photo of a television",
          "a photo of a tablet device",
          "a photo of test electronics",
          "a photo of tin containers",
          "a photo of large metal junk",
          "a photo of medium metal junk",
          "a photo of small metal junk",
          "a photo of a top-load washing machine",
          "a photo of a treadmill",
          "a photo of scrap tires",
          "a photo of electrical wires",
          "a photo of fitting wire scrap",
          "a photo of mixed wire scrap",
          "a photo of raw aluminum",
          "a photo of a toy car",
          "a photo of mixed brass and steel scrap",
          "a photo of a kitchen chimney hood",
          "a photo of a traditional stove scrap",
          "a photo of worn-out clothes",
          "a photo of an old computer cabinet",
          "a photo of torn paper pieces",
          "a photo of a double-drum washing machine",
          "a photo of a DVD case",
          "a photo of a refrigerator",
          "a photo of a fridge with damaged compressor",
          "a photo of an old fridge",
          "a photo of a damaged refrigerator",
          "a photo of a front-load washing machine",
          "a photo of a broken refrigerator door",
          "a photo of cracked glass",
          "a photo of a broken glass bottle",
          "a photo of miscellaneous items",
          "a photo of a broken cooler",
          "a photo of computer waste (motherboard, CPU, power supply)",
          "a photo of a computer keyboard",
          "a photo of metal household appliances",
          "a photo of small metal appliances",
          "a photo of a compact car",
          "a photo of a mini refrigerator",
          "a photo of a computer monitor",
          "a photo of a smartphone",
          "a photo of an unknown object (Pilthoni)",
          "a photo of plastic e-waste",
          "a photo of a plastic remote control",
          "a photo of broken plastic buttons",
          "a photo of a plastic speaker",
          "a photo of a plastic toy",
          "a photo of a general refrigerator",
          "a photo of an RO pump or motor",
          "a photo of a broken solar panel",
          "a phot of polythene bags",
          "a phot of polythene bag of scrap",
          "a photo of a power bank",
          "a photo of a sack of scrap",
          "a photo of a speaker box",
          "a photo of a Bluetooth speaker",
          "a photo of a sound system",
          "a photo of a split air conditioner",
          "a photo of a 1.5-ton Split AC",
          "a photo of a split air conditioning unit",
          "a photo of an old stabilizer",
          "a photo of steel scrap",
          "a photo of broken steel items",
          "a photo of tin scrap",
          "a photo of a traditional stove (Chulha) scrap",
          "a photo of a lathe or trade machine",
          "a photo of a commercial washing machine",
          "a photo of a water cooler",
          "a photo of air cooler",
          "a photo of an X-ray machine"
      ]

    # Initialize classifier
    classifier = EnsembleClassifier()

    # Load CSV
    csv_path = '/content/firebase_items_export.csv'
    print(f"Loading CSV from: {csv_path}")

    try:
        df = pd.read_csv(csv_path)
        print(f"CSV loaded. Shape: {df.shape}")
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return

    # Add new columns
    df['predicted_class'] = None
    df['confidence'] = 0.0
    df['ensemble_details'] = None
    df['blip_description'] = None
    df['detection_method'] = None

    print("Starting object detection and ensemble classification...")

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
        image_url = None
        if 'imageUrls' in df.columns and pd.notna(row['imageUrls']):
            value = str(row['imageUrls']).strip()
            if value.startswith('http'):
                image_url = value

        if image_url:
            predicted_class, confidence, model_predictions, blip_description = classifier.process_image(
                image_url, pickup_classes
            )

            if predicted_class:
                df.at[idx, 'predicted_class'] = predicted_class
                df.at[idx, 'confidence'] = confidence
                df.at[idx, 'ensemble_details'] = f"{len(model_predictions)} models used"
                df.at[idx, 'blip_description'] = blip_description
                df.at[idx, 'detection_method'] = 'object_detection' if model_predictions else 'full_image'
                print(f"Row {idx}: {predicted_class} (confidence: {confidence:.3f}) [{len(model_predictions)} models]")
            else:
                print(f"Row {idx}: Classification failed")
        else:
            print(f"Row {idx}: No valid image URL found in 'imageUrls' column")

    # Save results
    output_dir = '/content/drive/My Drive/similar_objects'
    os.makedirs(output_dir, exist_ok=True)

    output_file = os.path.join(output_dir, '2_classified_products_with_detection.csv')
    df.to_csv(output_file, index=False)

    print(f"\nResults saved to: {output_file}")

    # Detailed summary
    successful = df['predicted_class'].notna().sum()
    total = len(df)

    print(f"\nDetailed Summary:")
    print(f"Total products: {total}")
    print(f"Successfully classified: {successful}")
    print(f"Failed classifications: {total - successful}")
    print(f"Success rate: {successful/total*100:.1f}%")

    if successful > 0:
        avg_confidence = df[df['predicted_class'].notna()]['confidence'].mean()
        print(f"Average confidence: {avg_confidence:.3f}")

        # Confidence distribution
        high_conf = (df['confidence'] > 0.7).sum()
        med_conf = ((df['confidence'] > 0.4) & (df['confidence'] <= 0.7)).sum()
        low_conf = ((df['confidence'] > 0) & (df['confidence'] <= 0.4)).sum()

        print(f"\nConfidence Distribution:")
        print(f"High confidence (>0.7): {high_conf}")
        print(f"Medium confidence (0.4-0.7): {med_conf}")
        print(f"Low confidence (0-0.4): {low_conf}")

        print(f"\nTop 10 predicted classes:")
        top_classes = df['predicted_class'].value_counts().head(10)
        for class_name, count in top_classes.items():
            avg_conf_class = df[df['predicted_class'] == class_name]['confidence'].mean()
            print(f"  {class_name}: {count} (avg conf: {avg_conf_class:.3f})")

        # Detection method distribution
        detection_methods = df['detection_method'].value_counts()
        print(f"\nDetection methods used:")
        for method, count in detection_methods.items():
            print(f"  {method}: {count}")

if __name__ == "__main__":
    main()

Using device: cuda
Loading ensemble of models...
Loading OpenAI CLIP ViT-B/32...


100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 307MiB/s]


✓ OpenAI CLIP ViT-B/32 loaded
Loading OpenAI CLIP ViT-L/14...


100%|███████████████████████████████████████| 890M/890M [00:18<00:00, 50.1MiB/s]


✓ OpenAI CLIP ViT-L/14 loaded
Loading OpenCLIP ViT-B-32 (laion2b_s34b_b79k)...


...

Processing: 100%|█████████▉| 2511/2512 [2:12:24<00:02,  2.36s/it]

Row 2510: a photo of a traditional stove (Chulha) scrap (confidence: 0.282) [4 models]
No objects detected, classifying whole image


Processing: 100%|██████████| 2512/2512 [2:12:28<00:00,  3.16s/it]

Row 2511: a photo of a used inverter battery (confidence: 0.113) [4 models]



Results saved to: /content/drive/My Drive/similar_objects/2_classified_products_with_detection.csv

Detailed Summary:
Total products: 2512
Successfully classified: 1598
Failed classifications: 914
Success rate: 63.6%
Average confidence: 0.228

Confidence Distribution:
High confidence (>0.7): 40
Medium confidence (0.4-0.7): 111
Low confidence (0-0.4): 1447

Top 10 predicted classes:
  a photo of a used inverter battery: 658 (avg conf: 0.171)
  a photo of a traditional stove (Chulha) scrap: 182 (avg conf: 0.284)
  a photo of a power stabilizer: 150 (avg conf: 0.116)
  a phot of polythene bag of scrap: 85 (avg conf: 0.273)
  a photo of a computer keyboard: 59 (avg conf: 0.675)
  a photo of an unknown object (Pilthoni): 40 (avg conf: 0.222)
  a phot of polythene bags: 39 (avg conf: 0.313)
  a photo of plastic e-waste: 38 (avg conf: 0.189)
  a photo of duplex cardboard: 34 (avg conf: 0.346)
  a photo of a fridge with damaged compressor: 25 (avg conf: 0.182)

Detection methods used:
  objec